Import Libraries

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from dotenv import load_dotenv
import os
import sys
from pyspark.sql.functions import *

Set Java Home & Vars

In [6]:
#set java home
os.environ["JAVA_HOME"] = "C:\Program Files\Eclipse Adoptium\jdk-11.0.27.6-hotspot"

Set Spark Configs details

In [7]:
conf = SparkConf() \
    .setAppName("ETLPipeline") \
        .setMaster("local") \
            .set("spark.jars","C:\spark-3.5.6\jars\*")

Initiate Spark Session

In [8]:
sc = SparkContext.getOrCreate(conf=conf)
etl = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

Set Database details

In [ ]:
#get password from environment var
pwd = os.environ['PGPASSWORD']
uid = os.environ['PGUID']
#sql db details
server = "localhost"
src_db = "AdventureWorksDW2019"
target_db = "AdventureWorks"
src_driver = "org.postgresql.Driver"
target_driver = "org.postgresql.Driver"

In [ ]:
# source connection URL without user/password in query string
src_url = f"jdbc:postgresql://{server}:5432/{src_db}"

# target connection URL similarly
target_url = f"jdbc:postgresql://{server}:5432/{target_db}"

SQL Statement and test

In [ ]:
table_names = ['DimProduct', 'DimProductSubcategory', 'DimProductCategory', 'DimSalesTerritory', 'FactInternetSales']

dfs_list = []

for table in table_names:
    df = etl.read \
        .format("jdbc") \
        .option("url", src_url) \
        .option("dbtable", f'"public"."{table}"') \
        .option("user", uid) \
        .option("password", pwd) \
        .option("driver", "org.postgresql.Driver") \
        .load()
    dfs_list.append(df)

# Show the first table's data
dfs_list[0].show()

Py4JJavaError: An error occurred while calling o594.load.
: org.postgresql.util.PSQLException: FATAL: database "AdventureWorksDW2019" does not exist
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2734)
	at org.postgresql.core.v3.QueryExecutorImpl.readStartupMessages(QueryExecutorImpl.java:2859)
	at org.postgresql.core.v3.QueryExecutorImpl.<init>(QueryExecutorImpl.java:178)
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:344)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:57)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:277)
	at org.postgresql.Driver.makeConnection(Driver.java:448)
	at org.postgresql.Driver.connect(Driver.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProv

In [ ]:
data_collect = dfs.collect()
#looping through each row of the dataframe
for row in data_collect:
    #while looping through each
    #row printing the data of table_name
    print(row["table_name"])

NameError: name 'dfs' is not defined

Function to Extract source system data

In [ ]:
def extract():
    try:
        # Note: use .option() (singular) not .options() for each option
        dfs = etl.read \
            .format("jdbc") \
            .option("driver", src_driver) \
            .option("user", uid) \
            .option("password", pwd) \
            .option("url", src_url) \
            .option("query", sql) \
            .load()

        data_collect = dfs.collect()
        
        for row in data_collect:
            tbl_name = row["table_name"]
            print(f"Processing table: {tbl_name}")

            df = etl.read \
                .format("jdbc") \
                .option("driver", src_driver) \
                .option("user", uid) \
                .option("password", pwd) \
                .option("url", src_url) \
                .option("dbtable", f"dbo.{tbl_name}") \
                .load()

            df.show(10)

            load(df, tbl_name)  # Assuming you have a load() function defined

            print(f"Data loaded successfully from source table: {tbl_name}")

    except Exception as e:
        print(f"Error in extract: {str(e)}")


Function to persist data in target db

In [ ]:
def load(df, tbl):
    try:
        rows_imported = df.count()
        print(f"Importing {rows_imported} rows to table src_{tbl}...")

        df.write.mode("overwrite") \
            .format("jdbc") \
            .option("driver", target_driver) \
            .option("url", target_url) \
            .option("dbtable", "src_" + tbl) \
            .option("user", uid) \
            .option("password", pwd) \
            .save()

        print("Data loaded successfully to target table")

    except Exception as e:
        print(f"Error in load: {str(e)}")


In [ ]:
#function call
extract()